# Summary

**Public link to notebook: https://www.kaggle.com/jinyangpeh/generate-submissions-ipynb**

This notebook contains the implementation code to generate the submission.csv file and was originally ran on Kaggle.

All necessary code can be found in the accompanying datasets made public with this notebook. To generate the submission, simply run all the cells. The predicted semantic maps and submission.csv file can be found in the **Output**.

In [1]:
pip install patchify






Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..', 'input', 'cbam-unet'))
sys.path.append(os.path.join(os.getcwd(), '..', 'input', 'road-segmentation-data-loader'))
sys.path.append(os.path.join(os.getcwd(), '..', 'input', 'cbamunet-submission-generation'))

from cbam_unet import *
from load_data import *
from mask_to_submission import *

# %load_ext autoreload
# %autoreload 2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print(tf.version.VERSION)
print(tf.config.list_physical_devices('GPU'))

# Prevent automatic GPU memory pre-allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

2.13.0

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
# Directory paths
train_imgs_dir = os.path.join(os.path.dirname(os.getcwd()), 'input', 'road-segmentation-ds', 'training', 'images')
train_gts_dir = os.path.join(os.path.dirname(os.getcwd()), 'input', 'road-segmentation-ds', 'training', 'groundtruth')
test_imgs_dir = os.path.join(os.path.dirname(os.getcwd()), 'input', 'road-segmentation-ds', 'test', 'image')

In [5]:
def get_imgs_from_dir(img_dir):
    
    '''Returns a tf dataset from the directory of images'''
    imgs = []
    for i in range(1,51):
        img_fp = 'test_' + '%.3d' % (i)  + '.png'
        img_filename = os.path.join(test_imgs_dir, img_fp)
        imgs.append(img_filename)

    imgs_ds = tf.data.Dataset.from_tensor_slices(imgs)
    imgs_ds = imgs_ds.map(load_image)
    
    return imgs_ds

In [6]:
# Get class weights
road_pixel_prop, bg_pixel_prop = get_class_weights(train_imgs_dir, train_gts_dir)
ROAD_WEIGHT = 1./road_pixel_prop
BG_WEIGHT = 1./bg_pixel_prop

# Load in test images
test_imgs_ds = get_imgs_from_dir(test_imgs_dir)
len_before_patch = len(test_imgs_ds)

# Generate patches for test images
test_imgs_patched_ds = generate_patches_imgs(test_imgs_ds, (128, 128), 0, True)
len_after_patch = len(test_imgs_patched_ds)

# Determine number of patches per image
num_patch_per_image = len_after_patch/len_before_patch

# Batch patched dataset baed on number of patches per image to batch per image
test_imgs_patched_ds = test_imgs_patched_ds.batch(int(num_patch_per_image))

test_imgs_patched = list(iter(test_imgs_patched_ds))

In [7]:
def generate_submissions_patched():
    
    # Get class weights
    road_pixel_prop, bg_pixel_prop = get_class_weights(train_imgs_dir, train_gts_dir)
    ROAD_WEIGHT = 1./road_pixel_prop
    BG_WEIGHT = 1./bg_pixel_prop

    # Load in test images
    test_imgs_ds = get_imgs_from_dir(test_imgs_dir)
    len_before_patch = len(test_imgs_ds)
    
    # Generate patches for test images
    test_imgs_patched_ds = generate_patches_imgs(test_imgs_ds, (128, 128), 0, True)
    len_after_patch = len(test_imgs_patched_ds)
    
    # Determine number of patches per image
    num_patch_per_image = len_after_patch/len_before_patch
    
    # Batch patched dataset baed on number of patches per image to batch per image
    test_imgs_patched_ds = test_imgs_patched_ds.batch(int(num_patch_per_image))
    
    # Get original dimensions of the image
    for img in test_imgs_ds:
        original_shape = img.shape
        break
    original_height, original_width = original_shape[0], original_shape[1] 
    original_channel = 2
    
    # Instantiate model and load weights
    input_shape = (None, 128, 128, 3)
    model = CBAM_UNet(input_shape, BG_WEIGHT, ROAD_WEIGHT, 'weighted_iou')
    inputs = tf.keras.Input(shape=input_shape[1:])
    model.call(inputs)
    
    model.built = True
    weights_filepath = os.path.join(os.path.dirname(os.getcwd()), 'input', 'cbamunet-submission-generation', 'trial5_weights.h5')
    model.load_weights(weights_filepath)
    
    # Pass images through model and reconstruct full size semantic map
    reconstructed_imgs = np.zeros(shape = (len_before_patch, original_height, original_width, original_channel), dtype = np.float32)
    test_imgs_patched = list(iter(test_imgs_patched_ds))
    
    for img_no, img_patches in enumerate(test_imgs_patched):
        output = model(img_patches)
        reconstructed_imgs[img_no] = reconstruct_image(output, (128,128,2), (original_height, original_width, original_channel), 0, padding = True)
        
    test_output_filenames = []
    # Convert the numpy array pixel values into images
    for img_no, img_arr in enumerate(reconstructed_imgs):
        img_arr = np.argmax(img_arr, axis = -1)
        img_arr = (img_arr * 255).astype(np.uint8)
        img = Image.fromarray(img_arr, mode = 'L')
        output_filename = 'satImage_' + '%.3d' % (img_no + 1)  + '.png'
        img.save(output_filename)
        test_output_filenames.append(f'{os.getcwd()}/{output_filename}')
        
    # Use semantic maps to generate submission csv file    
    submission_filename = 'submission.csv'
    masks_to_submission(submission_filename, *test_output_filenames)

In [8]:
# To generate submissions using patched dataset
generate_submissions_patched()